# Cluster single -omics

Authors: Casper de Visser (casper.devisser@radboudumc.nl)

In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
from sklearn.cluster import spectral_clustering
from sklearn.metrics import v_measure_score
import snf
from snf import metrics
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels
import muon
import itertools

## Read in MuData object

In [ ]:
mdata = muon.read(mudata_object)

## Affinity networks for single -omics

In [ ]:
# Functions


def sort_fused_network(fused_network, labels_array):
    # Make Pandas Dataframes
    df = pd.DataFrame(fused_network)
    df_labels = pd.DataFrame(labels_array)
    df_labels.columns = ["Label"]
    # sort label df
    df_labels = df_labels.sort_values(by=['Label'])
    # sort fused network df with sorted labels
    df = df.reindex(df_labels.index)
    df = df[df_labels.index]
    array = df.to_numpy()
    np.fill_diagonal(array, 0)
    return(array)

def make_heatmap(array, n_clusters, data_types):
    # Create heatmap
    heatmap = plt.imshow(array, cmap='hot', interpolation='nearest')

    # Set axis names, title etc.
    plt.xlabel('samples')
    plt.ylabel('samples') 
    cbar = plt.colorbar(heatmap)
    cbar.ax.set_ylabel('sample correlations', loc="top")
    plt.suptitle(str(str(data_types) + '\nNumber of clusters: {:.2f}'.format(round(n_clusters)))) 
    plt.show()

    return(plt)  

In [ ]:
for i in mdata.mod.keys():

    # Make affinity matrix
    single_omics_affinity = snf.make_affinity(mdata[i].X, metric = 'euclidean', K=6, mu=0.5)

    # Determine optimal number of clusters (estimated via an eigengap approach)
    best, second = snf.get_n_clusters(single_omics_affinity)

    #Perform spectral clustering on the fused network
    labels = spectral_clustering(single_omics_affinity, n_clusters=best)
    
    #Sort Fused networks according to labels found by spectral clustering
    sorted_fused_network_best = sort_fused_network(single_omics_affinity, labels)

    make_heatmap(sorted_fused_network_best,  best, str(i))